In [1]:
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import os

In [2]:
KITTI360_base = '/data/KITTI-360'

image_dir = os.path.join(KITTI360_base, 'data_2d_raw')
pose_dir  = os.path.join(KITTI360_base, 'data_poses')
calib_dir = os.path.join(KITTI360_base, 'calibration')
pc_dir    = os.path.join(KITTI360_base, 'data_3d_raw')

sequence_names = os.listdir(pose_dir)
sequence_names.sort()

np.random.seed(0)
random_permutation = np.random.permutation(range(len(sequence_names)))
split_ratio = 0.8
validation_gaps = 20
num_train_split = int(split_ratio * len(sequence_names))
train_split = random_permutation[0:num_train_split]
val_split   = random_permutation[num_train_split:]

In [4]:
train_split, val_split, sequence_names

(array([7, 2, 1, 4, 8, 6, 3]),
 array([0, 5]),
 ['2013_05_28_drive_0000_sync',
  '2013_05_28_drive_0002_sync',
  '2013_05_28_drive_0003_sync',
  '2013_05_28_drive_0004_sync',
  '2013_05_28_drive_0005_sync',
  '2013_05_28_drive_0006_sync',
  '2013_05_28_drive_0007_sync',
  '2013_05_28_drive_0009_sync',
  '2013_05_28_drive_0010_sync'])

In [6]:
train_file = 'kitti360_train.txt'

train_lines = []

for sequence_index in train_split:
    sequence_name = sequence_names[sequence_index]
    pose_file = os.path.join(pose_dir, sequence_name, 'cam0_to_world.txt')
    key_frame_indexes = []
    with open(pose_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            key_frame_indexes.append(int(line.strip().split(" ")[0]))
            
    for i in range(1, len(key_frame_indexes)-1):
        image_dir_0 = os.path.join(image_dir, sequence_name, 'image_00', 'data_rect')
        image_paths = [
            os.path.join(image_dir_0, f"{i:010d}.png") for i in [key_frame_indexes[i], key_frame_indexes[i-1], key_frame_indexes[i+1]]
        ]
        image_exist=True
        for img in image_paths:
            if not os.path.isfile(img):
                image_exist=False
        if image_exist:
            wline = f"{sequence_name},{i},{key_frame_indexes[i]},{key_frame_indexes[i-1]},{key_frame_indexes[i+1]}\n"
            train_lines.append(wline)
        

with open(train_file, 'w') as f:
    f.writelines(train_lines)

val_lines   = []
val_file   = 'kitti360_val.txt'

for scene_index in val_split:
    sequence_name = sequence_names[scene_index]
    pose_file = os.path.join(pose_dir, sequence_name, 'cam0_to_world.txt')
    key_frame_indexes = []
    with open(pose_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            key_frame_indexes.append(int(line.strip().split(" ")[0]))
            
    for i in range(1, len(key_frame_indexes)-1, validation_gaps):
        image_dir_0 = os.path.join(image_dir, sequence_name, 'image_00', 'data_rect')
        image_paths = [
            os.path.join(image_dir_0, f"{i:010d}.png") for i in [key_frame_indexes[i], key_frame_indexes[i-1], key_frame_indexes[i+1]]
        ]
        image_exist=True
        for img in image_paths:
            if not os.path.isfile(img):
                image_exist=False
        if image_exist:
            wline = f"{sequence_name},{i},{key_frame_indexes[i]},{key_frame_indexes[i-1]},{key_frame_indexes[i+1]}\n"
            val_lines.append(wline)
        
with open(val_file, 'w') as f:
    f.writelines(val_lines)